In [6]:
from iohub.ngff import open_ome_zarr
from natsort import natsorted
from glob import glob
from pathlib import Path 
import torch
from torch.utils.data import Dataset


zarr_dir = "/mnt/efs/dlmbl/G-et/data/neuromast/Dataset/"
# defines input zarr file name with the zarr file structure
zarr_file = 'structured_celltype_classifier_data.zarr/*/*/*'
position_paths = natsorted(glob(zarr_dir + zarr_file))
# print(position_paths)


In [4]:
print("Number of positions: ", len(position_paths))

Number of positions:  1000


In [16]:
dataset = open_ome_zarr(position_paths[0], mode="r")
dataset.data.shape
all_chan = dataset.channel_names
chan = 'celltypes'
all_chan.index(chan)

2

In [3]:
class NeuromastDataset(Dataset):
    def __init__(self, file_path, class_channel: str = 'celltypes'):
        self.file_path = file_path
        #get the class index
        dataset = open_ome_zarr(self.file_path[0], mode="r")
        all_chan = dataset.channel_names
        self.c = all_chan.index(class_channel)
        

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, idx):
        dataset = open_ome_zarr(position_paths[idx], mode="r")
        image = dataset.data[:,0:self.c,:,:,:]
        label = dataset.data[:,self.c:self.c+1,:,:,:]
        return image, label

        


In [21]:
neuromast = NeuromastDataset(position_paths, class_channel='celltypes')
print(neuromast[0][0].shape)
print(neuromast[0][1].shape)

(1, 3, 73, 1024, 1024)
(1, 1, 73, 1024, 1024)


In [7]:
import numpy as np
from scipy.ndimage.morphology import binary_dilation
dataset = open_ome_zarr(position_paths[0], mode="r")
dataset.data.shape
all_chan = dataset.channel_names
chan = 'celltypes'
c = all_chan.index(chan)

image = dataset.data[:,0:c,:,:,:]
label = dataset.data[:,c:c+1,:,:,:]
segments = dataset.data[:,2]


unique = np.unique(segments)
print(unique)
segment_sizes = np.sum(segments, axis=(1, 2, 3, 4))
# print(segment_sizes)
# dilated_segments = binary_dilation(segments)
# segment_sizes = []
# for segment in segments:
#     size = np.sum(segment)
#     segment_sizes.append(size)
print(segment_sizes)
# print(dilated_segments)

/tmp/ipykernel_97688/774659179.py:2: DeprecationWarning: Please import `binary_dilation` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.morphology import binary_dilation


[ 0.  1.  2.  5.  8.  9. 10. 15. 16. 17. 18. 19. 24. 25. 26. 27. 28. 29.
 30. 31. 34. 35. 36. 37. 38. 41. 42. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 57. 60. 61. 62. 63. 64. 65. 66. 67. 68. 70. 71.]


AxisError: axis 4 is out of bounds for array of dimension 4